In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.6/615.6 kB 10.3 MB/s eta 0:00:00


In [2]:
from ultralytics import YOLO

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torchvision import models
from torch.nn.utils import prune
from torch.utils.data import DataLoader

In [4]:
import time
from tqdm.notebook import tqdm

In [5]:
model = YOLO('yolov8l.pt')

100%|██████████| 83.7M/83.7M [00:00<00:00, 312MB/s]


In [17]:
tensors = model.state_dict()
for name in tensors:
  print(name)
  print(tensors[name])

Streaming output truncated to the last 5000 lines.

         [[-1.1492e-03, -3.4180e-03,  1.2026e-03],
          [-4.9477e-03, -3.8643e-03, -1.5039e-03],
          [ 8.0299e-04,  2.1057e-03, -4.5276e-04]]],


        ...,


        [[[ 1.1740e-03,  8.7929e-04,  1.4400e-03],
          [-4.9782e-04, -2.9125e-03, -1.0931e-04],
          [-1.3447e-03, -3.2940e-03, -2.2182e-03]],

         [[ 2.4071e-03,  5.1613e-03,  4.3564e-03],
          [ 6.5804e-04,  1.1396e-03,  3.5973e-03],
          [-3.4714e-03, -4.0970e-03,  3.2234e-04]],

         [[ 2.6512e-03, -3.2139e-03, -2.6894e-03],
          [ 2.2411e-03, -1.3380e-03,  3.2449e-04],
          [ 1.7262e-03, -1.8587e-03, -2.0275e-03]],

         ...,

         [[ 7.2174e-03, -5.2118e-04, -3.3340e-03],
          [ 3.4447e-03, -1.5488e-03, -1.8883e-03],
          [-3.0637e-04, -1.5802e-03, -4.1783e-05]],

         [[ 2.7905e-03,  2.7180e-03, -1.0908e-04],
          [ 4.5753e-04,  1.2493e-03, -1.0090e-03],
          [-1.5364e-03, -1.7748e-03, -3

In [32]:
model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(320, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_r

In [58]:
from sklearn.cluster import kmeans_plusplus

tensors = model.state_dict()
for name in tensors:
  if name.find('weight') < 0:
    continue
  print(name)
  n = np.prod(tensors[name].shape)
  centroids = kmeans_plusplus(np.array(tensors[name].reshape(n, 1), dtype=float), 10)[0].squeeze()
  arr = tensors[name].reshape(n,)
  print(arr)
  print(np.unique(arr))
  for i in range(n):
    x_center = centroids[0]
    for center in centroids:
      if abs(arr[i] - center) < abs(arr[i] - x_center):
        x_center = center
    arr[i] = x_center
  print(arr)
  tensors[name] = arr.reshape(tensors[name].shape)
torch.save(tensors, 'clustered.pt')

model.model.0.conv.weight
tensor([-0.0651, -0.0651, -0.0651,  ..., -0.0651, -0.0651, -0.0651])
[  -0.065125]
tensor([-0.0651, -0.0651, -0.0651,  ..., -0.0651, -0.0651, -0.0651])
model.model.0.bn.weight
tensor([3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602,
        3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602])
[     3.1602]
tensor([3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602, 3.1602,

KeyboardInterrupt: ignored

In [62]:
torch.save(tensors, 'clustered.pt')

In [66]:
model = YOLO('yolov8l.pt')
model.load_state_dict(torch.load('clustered.pt'))

<All keys matched successfully>

In [68]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 166.848MB


In [69]:
inp = torch.randn(1, 3, 224, 224)

num_samples = 100
start_time = time.time()
for _ in tqdm(range(num_samples)):
    output = model(inp)
end_time = time.time()

infer_time = ((end_time - start_time) / num_samples) * 1000
print(f'Avg inference time: {infer_time:.4f} ms')

  0%|          | 0/100 [00:00<?, ?it/s]


WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.112935543060303. Dividing input by 255.
0: 224x224 1 potted plant, 428.5ms
Speed: 0.0ms preprocess, 428.5ms inference, 8.8ms postprocess per image at shape (1, 3, 224, 224)

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.112935543060303. Dividing input by 255.
0: 224x224 1 potted plant, 356.0ms
Speed: 0.0ms preprocess, 356.0ms inference, 1.5ms postprocess per image at shape (1, 3, 224, 224)

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.112935543060303. Dividing input by 255.
0: 224x224 1 potted plant, 370.3ms
Speed: 0.0ms preprocess, 370.3ms inference, 1.6ms postprocess per image at shape (1, 3, 224, 224)

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.112935543060303. Dividing input by 255.
0: 224x224 1 potted plant, 393.0ms
Speed: 0.0ms preprocess, 393.0ms inference, 1.4ms postprocess per image at shape (1, 3

Avg inference time: 473.5643 ms
